# 📊 Hotel Booking Analysis - Tuqa

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

In [ ]:

df_raw = pd.read_csv("first inten project (1).csv")
df_raw.columns = df_raw.columns.str.strip()
df = df_raw.copy()

In [ ]:

na = df.isnull().mean() * 100
na_to_plot = na[na > 0].sort_values(ascending=False)

if not na_to_plot.empty:
    na_to_plot.plot(kind='bar', figsize=(10,5))
    plt.title("Missing Values (%) - Before Cleaning")
    plt.ylabel("Percent")
    plt.tight_layout()
    plt.show()
else:
    print("✅ There are no columns with missing values before cleaning.")

In [ ]:

print("Unique values in 'booking status' before cleaning:", df['booking status'].unique())
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
sns.countplot(x='booking status', data=df_raw, ax=axs[0])
axs[0].set_title("Before Cleaning")
sns.countplot(x='booking status', data=df, ax=axs[1])
axs[1].set_title("After Cleaning")
plt.tight_layout()
plt.show()

In [ ]:

print("Columns before dropping:", df.columns.tolist())
df.drop_duplicates(inplace=True)
df.drop(columns=['Booking_ID', 'customer_id'], errors='ignore', inplace=True)
print("Columns after dropping:", df.columns.tolist())

In [ ]:

for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip()

In [ ]:

valid_status = ['Canceled', 'Not_Canceled']
df = df[df['booking status'].isin(valid_status)]
print("Unique values in 'booking status' after filtering:", df['booking status'].unique())
df['booking status'] = df['booking status'].map({'Canceled': 1, 'Not_Canceled': 0})
print("Missing values in 'booking status' after mapping:", df['booking status'].isnull().sum())

In [ ]:

numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove('booking status')

In [ ]:

df_raw[numeric_cols].hist(bins=30, layout=(3, 4), figsize=(15, 10))
plt.suptitle("Numeric Features - Before Cleaning", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    before = df.shape[0]
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    after = df.shape[0]
    print(f"Removed {before - after} outliers from '{col}'")

In [ ]:

df[numeric_cols].hist(bins=30, layout=(3, 4), figsize=(15, 10))
plt.suptitle("Numeric Features - After Removing Outliers", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:

skewed = df[numeric_cols].skew().sort_values(ascending=False)
skewed_features = skewed[abs(skewed) > 0.75].index
print("Features with high skewness:", list(skewed_features))

for col in skewed_features:
    if (df[col] > 0).all():
        df[col] = np.log1p(df[col])

In [ ]:

df[skewed_features].hist(bins=30, layout=(1, len(skewed_features)), figsize=(15, 5))
plt.suptitle("Skewed Features After Log Transform", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:

cat_cols = df.select_dtypes(include='object').columns.tolist()
print("Categorical columns to encode:", cat_cols)
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [ ]:

numeric_raw = df_raw.select_dtypes(include=np.number)
sns.heatmap(numeric_raw.corr(), cmap="coolwarm", center=0)
plt.title("Correlation Matrix - Before Cleaning")
plt.tight_layout()
plt.show()

sns.heatmap(df.corr(), cmap="coolwarm", center=0)
plt.title("Correlation Matrix - After Cleaning")
plt.tight_layout()
plt.show()

In [ ]:

corr = df.corr()
upper_triangle = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
high_corr = [col for col in upper_triangle.columns if any(abs(upper_triangle[col]) > 0.9)]
print("Highly correlated columns to drop:", high_corr)
df.drop(columns=high_corr, inplace=True)

In [ ]:

X = df.drop('booking status', axis=1)
y = df['booking status']

In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

sns.countplot(x=y_resampled)
plt.title("Booking Status Distribution After SMOTE")
plt.xticks([0, 1], ['Not_Canceled', 'Canceled'])
plt.tight_layout()
plt.show()

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("\n🎯 Accuracy:", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred))
print("\n📉 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
features = X.columns

sns.barplot(x=importances[indices][:15], y=features[indices][:15])
plt.title("Top 15 Feature Importances")
plt.tight_layout()
plt.show()